# Obtén datos de la API de Marvel

![imagen](https://cdn.hobbyconsolas.com/sites/navi.axelspringer.es/public/styles/hc_1440x810/public/media/image/2021/09/marvel-2460339.jpg?itok=cBnC1CBi)

Te tendrás que [registrar](https://developer.marvel.com/) y consultar la [documentación](https://developer.marvel.com/docs)

Queremos que consultes a la api para que te devuelva la info de los personajes de marvel que empiecen por la inicial de tu nombre. Deberemos guardarlos en un csv la información con la siguiente estructura

![imagen](./img/Captura_marvel.PNG)

Aquí te dejamos el código en python para poder empezar a hacer las llamadas, rellenando algunas variables como tus keys, parámetros de la llamada y la url (endpoint) a la que quieres acceder

In [1]:
# Importar las librerías necesarias
import hashlib  # Para crear el hash MD5 requerido por la API de Marvel
import requests  # Para hacer peticiones HTTP a la API
import datetime  # Para generar el timestamp
import pandas as pd  # Para manejar los datos en formato DataFrame
from dotenv import load_dotenv  # Para cargar variables de entorno desde archivo .env
import os  # Para acceder a variables de entorno

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

def hash_params(timestamp, priv_key, pub_key):
    """
    Función que genera el hash MD5 requerido por Marvel API
    Marvel requiere un hash que combina: timestamp + clave privada + clave pública
    """
    hash_md5 = hashlib.md5()
    hash_md5.update(f'{timestamp}{priv_key}{pub_key}'.encode('utf-8'))
    hashed_params = hash_md5.hexdigest()
    return hashed_params

# Generar timestamp actual para la autenticación
timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')

# Obtener las claves API desde las variables de entorno (más seguro que hardcodearlas)
pub_key = os.getenv('MARVEL_PUB_KEY')
priv_key = os.getenv('MARVEL_PRIV_KEY')

# Preparar los parámetros para la petición a la API
params = {'ts': timestamp,  # Timestamp para autenticación
        'apikey': pub_key,  # Clave pública
        'hash': hash_params(timestamp, priv_key, pub_key),  # Hash MD5 generado
        'nameStartsWith': "B",  # Filtrar personajes que empiecen por B
        'offset': 0,  # Empezar desde el primer resultado
        'limit': 100  # Máximo 100 resultados por petición (límite de Marvel API)
        }

# URL del endpoint de personajes de Marvel
url = 'https://gateway.marvel.com/v1/public/characters'

# Hacer la petición GET a la API
res = requests.get(url, params=params)

# Crear diccionario para almacenar los datos que nos interesan
marvel_dict = {"id": [],
               "name": [],
               "picture_url": []}

# Iterar sobre cada personaje en la respuesta y extraer sus datos
for personaje in res.json()['data']['results']:
    marvel_dict['id'].append(personaje['id'])  # ID único del personaje
    marvel_dict['name'].append(personaje['name'])  # Nombre del personaje
    # Construir URL completa de la imagen (path + extensión)
    marvel_dict['picture_url'].append(personaje['thumbnail']['path'] + "." + personaje['thumbnail']['extension'])

# Convertir el diccionario a DataFrame de pandas
df_1 = pd.DataFrame(marvel_dict)

# Guardar los datos en un archivo CSV
df_1.to_csv("marvel_B.csv", index=False)

# Mostrar cuántos personajes se obtuvieron
print(f"Total de personajes obtenidos: {len(df_1)}")
df_1

Total de personajes obtenidos: 100


,id,name,picture_url
0,1011456,Balder,http://i.annihil.us/u/prod/marvel/i/mg/b/f0/4c...
1,1009168,Banshee,http://i.annihil.us/u/prod/marvel/i/mg/b/03/52...
2,1009596,Banshee (Theresa Rourke),http://i.annihil.us/u/prod/marvel/i/mg/b/c0/4c...
3,1011778,Baron Mordo (Karl Mordo),http://i.annihil.us/u/prod/marvel/i/mg/f/90/4c...
4,1009169,Baron Strucker,http://i.annihil.us/u/prod/marvel/i/mg/8/80/4c...
...,...,...,...
95,1009211,Bucky,http://i.annihil.us/u/prod/marvel/i/mg/d/03/4d...
96,1011038,Bug,http://i.annihil.us/u/prod/marvel/i/mg/8/30/4c...
97,1010887,Bulldozer,http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...
98,1009212,Bullseye,http://i.annihil.us/u/prod/marvel/i/mg/6/30/53...


In [2]:
# VERSIÓN MEJORADA: Para cuando hay más de 100 personajes

import hashlib
import requests
import datetime
import pandas as pd
import os

def hash_params(timestamp, priv_key, pub_key):
    """
    Función que genera el hash MD5 requerido por Marvel API
    Marvel requiere un hash que combina: timestamp + clave privada + clave pública
    """
    hash_md5 = hashlib.md5()
    hash_md5.update(f'{timestamp}{priv_key}{pub_key}'.encode('utf-8'))
    hashed_params = hash_md5.hexdigest()
    return hashed_params

# Generar timestamp actual
timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')

# Obtener claves desde variables de entorno
pub_key = os.getenv('MARVEL_PUB_KEY')
priv_key = os.getenv('MARVEL_PRIV_KEY')

# Parámetros iniciales para la primera petición
params = {'ts': timestamp, 
        'apikey': pub_key, 
        'hash': hash_params(timestamp, priv_key, pub_key),
        'nameStartsWith': "B",  # Buscar personajes que empiecen por B
        'offset': 0,  # Comenzar desde el registro 0
        'limit': 100  # Pedir 100 resultados (máximo permitido por petición)
        }

# Endpoint de la API de Marvel
url = 'https://gateway.marvel.com/v1/public/characters'

# Primera petición a la API
res = requests.get(url, params=params)
res_json = res.json()

# Diccionario para almacenar todos los personajes
marvel_dict = {"id": [],
               "name": [],
               "picture_url": []}

# Procesar los primeros 100 resultados
for personaje in res_json['data']['results']:
    marvel_dict['id'].append(personaje['id'])
    marvel_dict['name'].append(personaje['name'])
    marvel_dict['picture_url'].append(personaje['thumbnail']['path'] + "." + personaje['thumbnail']['extension'])

# Crear DataFrame con los primeros resultados
df_1 = pd.DataFrame(marvel_dict)

# Obtener el total de personajes disponibles según la API
total = res_json['data']['total']

# BUCLE: Seguir pidiendo datos mientras no tengamos todos los personajes
while len(df_1) < total:
    print(f"Obteniendo más datos... {len(df_1)}/{total}")
    
    # Incrementar el offset para pedir los siguientes 100 personajes
    params['offset'] += 100
    
    # Nueva petición con el offset actualizado
    res = requests.get(url, params=params)
    res_json = res.json()
    
    # Agregar los nuevos personajes al diccionario
    for personaje in res_json['data']['results']:
        marvel_dict['id'].append(personaje['id'])
        marvel_dict['name'].append(personaje['name'])
        marvel_dict['picture_url'].append(personaje['thumbnail']['path'] + "." + personaje['thumbnail']['extension'])
    
    # Actualizar el DataFrame con todos los personajes acumulados
    df_1 = pd.DataFrame(marvel_dict)

# Guardar todos los personajes en un archivo CSV
df_1.to_csv("marvel_B.csv", index=False)

# Mostrar el total final de personajes obtenidos
print(f"Total de personajes obtenidos: {len(df_1)}")
df_1

Obteniendo más datos... 100/101
Total de personajes obtenidos: 101


,id,name,picture_url
0,1011456,Balder,http://i.annihil.us/u/prod/marvel/i/mg/b/f0/4c...
1,1009168,Banshee,http://i.annihil.us/u/prod/marvel/i/mg/b/03/52...
2,1009596,Banshee (Theresa Rourke),http://i.annihil.us/u/prod/marvel/i/mg/b/c0/4c...
3,1011778,Baron Mordo (Karl Mordo),http://i.annihil.us/u/prod/marvel/i/mg/f/90/4c...
4,1009169,Baron Strucker,http://i.annihil.us/u/prod/marvel/i/mg/8/80/4c...
...,...,...,...
96,1011038,Bug,http://i.annihil.us/u/prod/marvel/i/mg/8/30/4c...
97,1010887,Bulldozer,http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...
98,1009212,Bullseye,http://i.annihil.us/u/prod/marvel/i/mg/6/30/53...
99,1009213,Bushwacker,http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...
